In [1]:
import mlcv_py
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tabulate
from skimage.metrics import mean_squared_error, peak_signal_noise_ratio, structural_similarity

In [2]:
def imshow_bgr(img_bgr):
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    plt.gca().set_xticks([])
    plt.gca().set_yticks([])
    plt.imshow(img_rgb)
    plt.plot()

In [3]:
from pathlib import Path
import shutil

data_root = Path("../data/images")
out_root = Path("results")
if out_root.exists():
    shutil.rmtree(str(out_root))
out_root.mkdir()

SAMPLES_PER_CATEGORY = 10

In [4]:
JPEG_QUALITY_LEVELS = [i for i in range(0, 101, 5)]
MULTICUT_QUALITY_SETTINGS = [0.5, 1, 3, 5, 10, 20, 50, 75, 100, 250, 500, 750, 1000, 2000, 3000, 5000, 7500, 10000, 15000, 25000, 35000, 50000, 100000]

In [5]:
import random 
from tqdm import tqdm

for sub_dir in data_root.glob("*"):
    if not sub_dir.is_dir(): continue

    category_dir = out_root / sub_dir.stem
    category_dir.mkdir()

    img_paths = list(sub_dir.glob("*.png"))
    sample_paths = random.sample(img_paths, k=SAMPLES_PER_CATEGORY)
    for sample_path in tqdm(sample_paths):

        img = cv2.imread(sample_path, cv2.IMREAD_COLOR)
        sample_dir = category_dir / sample_path.stem
        sample_dir.mkdir()

        cv2.imwrite(str(sample_dir / "original.png"), img)

        for method in ["jpg", "multicut"]:
            qualities = JPEG_QUALITY_LEVELS if method == "jpg" else MULTICUT_QUALITY_SETTINGS
            for q in qualities:
                sample_basename = f"{sample_path.stem}-{method}-{q}"

                if method == "jpg":
                    res, data = cv2.imencode(".jpg", img, [cv2.IMWRITE_JPEG_QUALITY, q])
                    assert res
                    decoded_img = cv2.imdecode(data, cv2.IMREAD_COLOR)
                    out_name = f"{sample_basename}-{len(data)*8}.png"
                    out_path = sample_dir / out_name
                    cv2.imwrite(str(out_path), decoded_img)
                elif method == "multicut":
                    decoded_img, bits = mlcv_py.compress_grid_mean_huffman(img, weight_size=q)
                    out_name = f"{sample_basename}-{bits}.png"
                    out_path = sample_dir / out_name
                    cv2.imwrite(str(out_path), decoded_img)
                else:
                    raise ValueError("method does not exist")

                # print(sample_basename)




100%|██████████| 10/10 [01:52<00:00, 11.27s/it]
